# 深度学习计算4_读写和储存   

## 读写Tensor
可以直接使用`save`函数和`load`函数分别存储和读取Tensor。      

- **`save`**:使用Python的`pickle`实用程序将对象进行**序列化**，然后将序列化的对象保存到disk，使用`save`可以保存各种对象,包括模型、张量和字典等。    


- **`load`**:使用`pickle.unpickle`工具将`pickle`的对象文件反序列化为内存。

下面的例子创建了Tensor变量`x`，并将其存在文件名同为`x.pt`的文件里。

In [1]:
import torch
from torch import nn

x = torch.ones(3)
torch.save(x, 'x.pt')

x2 = torch.load('x.pt')
x2

tensor([1., 1., 1.])

可以存储一个`Tensor`列表并读回内存。

In [2]:
y = torch.zeros(4)
torch.save([x,y],"xy.pt")
xy_list = torch.load("xy.pt")
xy_list

[tensor([1., 1., 1.]), tensor([0., 0., 0., 0.])]

存储并读取一个从字符串映射到Tensor的字典。

In [3]:
torch.save({"x":x,"y":y},"xy_dict.pt")
xy = torch.load("xy_dict.pt")
xy

{'x': tensor([1., 1., 1.]), 'y': tensor([0., 0., 0., 0.])}

## 读写模型   

### `state_dict`
在PyTorch中，`Module`的可学习参数(即权重和偏差)，模块模型包含在参数中(通过`model.parameters()`访问)。`state_dict`是一个从参数名称隐射到参数`Tesnor`的字典对象。

In [4]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.hidden = nn.Linear(3, 2)
        self.act = nn.ReLU()
        self.output = nn.Linear(2, 1)

    def forward(self, x):
        a = self.act(self.hidden(x))
        return self.output(a)

net = MLP()
net.state_dict()

OrderedDict([('hidden.weight',
              tensor([[-0.0112,  0.2410, -0.5349],
                      [-0.4050, -0.3574, -0.3208]])),
             ('hidden.bias', tensor([0.4222, 0.2475])),
             ('output.weight', tensor([[-0.5794, -0.2376]])),
             ('output.bias', tensor([-0.4280]))])

- **注意**:只有具有**可学习参数的层**(卷积层、线性层等)才有`state_dict`中的条目。优化器(`optim`)也有一个`state_dict`，其中包含关于优化器状态以及所使用的超参数的信息。

In [5]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer.state_dict()

{'state': {},
 'param_groups': [{'lr': 0.001,
   'momentum': 0.9,
   'dampening': 0,
   'weight_decay': 0,
   'nesterov': False,
   'params': [0, 1, 2, 3]}]}

### 保存和加载模型

PyTorch中保存和加载训练模型有两种常见的方法:

1. 保存和加载模型参数——`state_dict`(推荐方式)

```
torch.save(model.state_dict(), PATH) # 推荐的文件后缀名是pt或pth
model = TheModelClass(*args, **kwargs)
model.load_state_dict(torch.load(PATH))

```

2. 保存和加载整个模型


In [8]:
# 保存和加载模型参数-state_dict
X = torch.randn(2, 3)
Y = net(X)

PATH = "./net.pt"
torch.save(net.state_dict(), PATH)

net2 = MLP()
net2.load_state_dict(torch.load(PATH))
Y2 = net2(X)
Y2 == Y

tensor([[True],
        [True]])

因为`net`和`net2`都有同样的模型参数，那么对同一个输入`X`的计算结果将会是一样的。